### Introduction

Tuning of a rooftop PV power site by using measurement data provided by the user to find the best estimates for capacity, azimuth, and tilt for the site. 

Requires power measurements in kW, and the latitude and longitude of the site. Optionally takes a user estimate for azimuth and tilt, but these should only be filled if the user has a high level of confidence in the value(s).

If power mesurements cover both summer and winter, parameter estimates will be more accurate. Data does not have to be continuous, the example given has data for January and July only.

This example also provides accuracy analysis throughout.

### 0: Imports and functions

In [3]:
#!pip install solcast plotly kaleido

In [1]:
from solcast.unmetered_locations import UNMETERED_LOCATIONS
stonehenge = UNMETERED_LOCATIONS["Stonehenge"]
sydney = UNMETERED_LOCATIONS["Sydney Opera House"]
SOLCAST_API_KEY = "gG8Ww216F5UzVYn5gm6ajBw5XlINl647"

In [2]:
import numpy as np
import pandas as pd
from scipy.optimize import minimize
from typing import Optional
import concurrent.futures

import solcast

In [6]:
def get_rooftop_data(
    latitude: float,
    longitude: float,
    params: dict,
    months: list,
    period: str,
    api_key: str,
) -> pd.DataFrame:
    """Runs multiple api calls for rooftop pv power with chunks of time
    as solcast.historic can only take 31 days at a time. Returns a
    concatenated df of the results"""
    futures = []
    with concurrent.futures.ThreadPoolExecutor(max_workers=2) as pool:
        for start, end in months:
            f = pool.submit(
                solcast.historic.rooftop_pv_power,
                latitude=latitude,
                longitude=longitude,
                **params,
                start=start,
                end=end,
                period=period,
                api_key=api_key,
            )
            futures.append(f)
    df = []
    for f in futures:
        res = f.result()
        if res.success:
            df.append(res.to_pandas())
        else:
            # NOTE for production purposes you will need to deal with API failures, e.g. due rate-limiting!
            pass

    df = pd.concat(df)
    return df


def generate_monthly_chunks(df_index: pd.DatetimeIndex) -> list:
    """Takes the index of the measurement data to get a start and end date for each chunk
    of time, as the solcast.historic function can only take 31 days per call. Returns
    (start,end) for each month."""
    chunks = []
    df_index = pd.to_datetime(df_index)
    unique_months = sorted(set(df_index.to_period("M")))

    for period in unique_months:
        month_start = period.start_time
        month_end = period.end_time

        # Filter the DataFrame index to get only the dates within this month
        month_data = df_index[(df_index >= month_start) & (df_index <= month_end)]

        if not month_data.empty:
            start_date = month_data.min().strftime("%Y-%m-%d")
            end_date = (month_data.max() + pd.Timedelta("24h")).strftime("%Y-%m-%d")
            chunks.append((start_date, end_date))

    return chunks


def grid_search_parameter_estimate(
    latitude: float,
    longitude: float,
    site_parameter_estimates: dict,
    meas: pd.DataFrame,
    pv_power_column_name: str,
    months: list,
    param: str,
    period: str,
    api_key: str,
    best_guess: list,
    display_results: Optional[bool] = None,
):
    """Pass in the existing metadata, measurement data and the parameter you want to estimate alongside a best guess.
    Searches for the best capacity with each parameter estimate, and optionally displays the MAE results for each parameter combo.
    First output is the best estimate for the parameter, second ouput is the estimated capacity for estimated capacity.
    """

    if display_results is None:
        display_results = False

    if not isinstance(best_guess, list):
        best_guess = [best_guess]

    allowed_parameters = ["azimuth", "tilt"]
    if not param in allowed_parameters:
        raise KeyError(
            f"Supplied parameter is not valid. Parameter must be one of the following {allowed_parameters}."
        )

    error_results = []
    param_combo = []
    site_copy = site_parameter_estimates.copy()

    for estimate in best_guess:
        site_copy[param] = estimate
        est = get_rooftop_data(
            latitude, longitude, site_copy, months, period, api_key
        ).tz_convert(None)

        # for each estimate, find the best capacity pairing.
        res = minimize(
            minimize_me,
            site_parameter_estimates["capacity"],
            args=(
                est["pv_power_rooftop"].loc[meas.index].values,
                meas[pv_power_column_name].values,
            ),
        )
        capacity_est = np.round(res.x[0] * site_parameter_estimates["capacity"], 0)
        error = res.fun
        param_combo.append((estimate, capacity_est))
        error_results.append({"capacity": capacity_est, "mae%": 100 * error})
    results_df = pd.DataFrame.from_records(error_results, index=best_guess)
    results_df.index.name = param
    results_df = results_df.sort_values(by="mae%")
    if display_results:
        display(results_df.round(1))

    # take the lowest error result from the capacity and parameter pairing
    est_param = results_df["mae%"].idxmin()
    est_capacity = results_df.loc[est_param, "capacity"]

    return est_param, est_capacity


def minimize_me(x, est, obs):
    """Calculates the MAE for given scenario to be used in scipy.optimize.minimize() for grid_search_parameter_estimate()"""
    error = np.mean(np.abs(est * x[0] - obs))
    return error

### 2a: Finding an Azimuth, Tilt, and Capacity Estimate

Same process for Azimuth and Tilt:

1. calculate_single_param runs a grid search for a wide range of values and finds the best capacity pairing. 
2. Then it runs another search around the pair with the lowest error and does a narrower search.

In [4]:
def calculate_parameters(initial_params: dict, latitude: float, longitude: float, measurement_data: pd.DataFrame, data_period: str, power_column_name: str) -> dict:

    months = generate_monthly_chunks(measurement_data.index)

    # required for all estimations
    generic_kwargs = {
        "latitude": latitude,
        "longitude": longitude,
        "months": months,
        "meas": measurement_data,
        "pv_power_column_name": power_column_name,
        "period": data_period,
        "api_key": SOLCAST_API_KEY,
    }

    site_parameter_estimates = {}
    # take an initial capacity estimate and show an initial estimation with no tuning
    capacity_initial_estimate = measurement_data[power_column_name].max()
    site_parameter_estimates["capacity"] = capacity_initial_estimate

    # run for azimuth
    site_parameter_estimates = calculate_single_param(generic_kwargs,
                                      estimate=site_parameter_estimates, 
                                      initial_min=initial_params["initial_azi_min"], 
                                      initial_max=initial_params["initial_azi_max"], 
                                      delta=initial_params["azi_delta"], 
                                      param="azimuth")
    
    # update for tilt
    site_parameter_estimates = calculate_single_param(generic_kwargs,
                                      estimate=site_parameter_estimates, 
                                      initial_min=initial_params["initial_tilt_min"], 
                                      initial_max=initial_params["initial_tilt_max"], 
                                      delta=initial_params["tilt_delta"], 
                                      param="tilt")
    
    return site_parameter_estimates
    

def calculate_single_param(generic_kwargs, estimate, initial_min: float, initial_max: float, delta: float, param: str) -> dict:
    # Initial wide search
    initial_search = np.arange(
        initial_min, initial_max + 1, delta
    ).tolist()
    first_guess, updated_capacity_est = grid_search_parameter_estimate(
        **generic_kwargs, site_parameter_estimates=estimate, param=param, best_guess=initial_search, display_results=False
    )

    # Secondary search around estimate from wide search
    secondary_search = np.arange(
        int(first_guess - delta / 2), int(first_guess + delta / 2), 2
    ).tolist()
    param_est, updated_capacity_est = grid_search_parameter_estimate(
        **generic_kwargs, site_parameter_estimates=estimate, param=param, best_guess=secondary_search, display_results=False
    )

    # Update params
    estimate["capacity"] = updated_capacity_est
    estimate[param] = param_est

    return estimate

In [5]:
initial_params = {
    "initial_azi_min": -90,
    "initial_azi_max": 90,
    "azi_delta": 30,
    "initial_tilt_min": 10,
    "initial_tilt_max": 50,
    "tilt_delta": 10,
}

API_KEY = SOLCAST_API_KEY
measurement_data = pd.read_csv(
    "https://solcast.github.io/solcast-api-python-sdk/notebooks/data/3.4_sample_measurements.csv",
    index_col=[0],
    parse_dates=True,
)  # Make sure this is in kW. Timestamps must be in UTC

power_column_name = (
    "pv_power_rooftop"  # Set this to the name of the power column in measurement_data
)

latitude = sydney["latitude"]
longitude = sydney["longitude"]
period = "PT60M"

new_estimates = calculate_parameters(
    initial_params=initial_params, 
    latitude=latitude, 
    longitude=longitude, 
    measurement_data=measurement_data, 
    data_period=period, 
    power_column_name=power_column_name
    )

print("Current Parameter Estimates:", new_estimates)

Azimuth Parameters: {'capacity': 8.0, 'azimuth': 32}
Full Parameters: {'capacity': 8.0, 'azimuth': 32, 'tilt': 27}
Current Parameter Updates: None


In [ ]:
res_forecast = solcast.forecast.rooftop_pv_power(
    api_key=SOLCAST_API_KEY,
    latitude=latitude, 
    longitude=longitude,
    output_parameters='pv_power_rooftop',
    capacity=new_estimates["capacity"],
    azimuth=new_estimates["azimuth"],
    tilt=new_estimates["tilt"]
)